In [514]:
import pandas as pd
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

Function 1: pluralize_gift()

Vectorized Function

In [515]:
def pluralize_gift(gift):
  """
  Creates a plural version of a noun.
  
  Parameters
  ----------
  gift: str
    A noun
    
  Return
  ------
  str
    Plural version 
  """

  if gift.endswith('y'):
    gift = gift.replace('y', 'ies')
  elif ("oo") in gift:
    gift = gift.replace('oo', 'ee')
  else:
    gift = gift + "s"

  return gift

In [516]:
#Unit test
pluralize_gift("goose")

'geese'

Function 2: make_phrase()

In [517]:
def make_phrase(num, num_word, item, verb, adjective, location):
    """
    Creates a phrase based on the input for a specific day of gifts.
    
    Parameters
    ----------
    num: int
        The day number (e.g., 1, 2, 3...)
    num_word: str
        The word form of the day number (e.g., "one", "two", "three...")
    item: str
        The gift item (e.g., "partridge", "lord", etc.)
    verb: str
        The action verb (e.g., "a-leaping", "a-swimming")
    adjective: str
        An optional adjective to describe the gift item (e.g., "golden", "french")
    location: str
        The optional location of the gift (e.g., "in a pear tree")
    
    Returns
    -------
    str
        The created phrase for that day's gift.
    """
    
    ## Step 1: Replace NAs with blank strings
    verb = "" if pd.isna(verb) else verb
    adjective = "" if pd.isna(adjective) else adjective
    location = "" if pd.isna(location) else location
    
    ## Step 2: If the day number is larger than 1, the gift items need pluralized!
    if num_word != "one":
      item = pluralize_gift(item)
    
    #ChatGPT helped me figure out if a gift starts with a vowel and replace the day with an "an" if it does and an "a" if it doesn't. 
    ## Step 3: Figure out if a gift item starts with a vowel
    vowels = "aeiou"
    starts_with_vowel = item[0].lower() in vowels
    
    # Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an" and if the gift item does not start with a vowel, replace the day with "a" (e.g. a partridge in a pear tree). If it is not the first day, use just the number word (e.g. ten lords a leap)
    if num == 1:
        if starts_with_vowel:
            num_word = "an"
        else:
            num_word = "a"
    
    # Step 5: Build the full phrase
    #ChatGPT helped me create the full_phrase by combining all the parts and helping me fix the spacing in the full_phrase.
    full_phrase = f"{num_word} {adjective} {item} {verb} {location}".strip()
    return " ".join(full_phrase.split()) 


In [518]:
#Unit test
make_phrase(num = 10,
            num_word = "ten", 
            item = "lord", 
            verb = "a-leaping", 
            adjective = "", 
            location = "")

'ten lords a-leaping'

In [519]:
#Unit test
make_phrase(5, "five", "ring", None, "golden", None)

'five golden rings'

Function 3: sing_day()

In [520]:
def sing_day(dataset, num, phrase_col):
    """
    Generates the song lyrics for the 'num' day of Christmas.
    
    Parameters
    ----------
    dataset: 
        A data structure containing the columns: Day, Day.in.Words, Gift.Item, Verb, Adjective, Location
    num: int
        The day number (e.g., 1, 2, 3...) 
    phrase_col: 
        The column name in the dataset where phrases are stored. 
    
    Returns
    -------
    str
        A string containing the lyrics for the specified day of Christmas.
    """
    
    day_word_map = {
    1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five',
    6: 'six', 7: 'seven', 8: 'eight', 9: 'nine', 10: 'ten',
    11: 'eleven', 12: 'twelve'}

    #ChatGPT helped me map day to the new words we created in the dictionary above.
    dataset['Day.Word'] = dataset['Day'].map(day_word_map)

    #Create new column called "Full.Phrase" in the dataset
    #ChatGPT helped me apply the make_phrase function to each row in the dataset and create a new column called "Full.Phrase"
    dataset['Full.Phrase'] = dataset.apply(
    lambda row: make_phrase(
        num=row['Day'],
        num_word=row['Day.Word'],  
        item=row['Gift.Item'],
        verb=row['Verb'],
        adjective=row['Adjective'],
        location=row['Location']
    ),
    axis=1  
)

    # Step 1: Setup the intro line
    number_to_word = {
        1: "first",
        2: "second",
        3: "third",
        4: "fourth",
        5: "fifth",
        6: "sixth",
        7: "seventh",
        8: "eighth",
        9: "ninth",
        10: "tenth",
        11: "eleventh",
        12: "twelfth"
    }
    
    #ChatGPT helped me map the num to num_word
    if num in number_to_word:
        num_word = number_to_word[num]
    else:
        return "Invalid day number"

    intro = "On the " + num_word + " day of Christmas, my true love sent to me: \n" 

    # Step 2: Sing the gift phrases
    gifts = ""
    
    #ChatGPT helped me add commas, new lines, and a period to the end of the final line. 
    #ChatGPT helped me iterate through the gifts in reverse order.
    for i in range(num, 0, -1):
        gift = dataset[phrase_col].iloc[i-1]

        if i == 1 and num > 1:
            gift = "and " + gift

        if i > 1:
            gift += ","

        gifts += gift + "\n"

    gifts = gifts.rstrip(",\n") + "."
        

    # Step 3: Put it all together and return
    #ChatGPT helped me combine the intro and gifts string and use the strip() method to get rid of any extra whitespace.
    return intro + gifts.strip()

In [521]:
#Unit test
print(sing_day(xmas, 3, "Full.Phrase"))

On the third day of Christmas, my true love sent to me: 
three french hens,
two turtle doves,
and a partridge in a pear tree.
On the third day of Christmas, my true love sent to me: 
three french hens,
two turtle doves,
and a partridge in a pear tree.


In [522]:
#Unit test
print(sing_day(xmas, 5, "Full.Phrase"))

On the fifth day of Christmas, my true love sent to me: 
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.
On the fifth day of Christmas, my true love sent to me: 
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.


In [523]:
#ChatGPT helped me apply the sing_day function through days 1 through 12 to print out the entire lyrics of the 12 Days of Christmas song with a seperation between each day. 
results = map(lambda day: sing_day(xmas, day, 'Full.Phrase'), range(1, 13))
results = list(results)
print("\n\n".join(results))

On the first day of Christmas, my true love sent to me: 
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me: 
two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love sent to me: 
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fourth day of Christmas, my true love sent to me: 
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fifth day of Christmas, my true love sent to me: 
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the sixth day of Christmas, my true love sent to me: 
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the seventh day of Christmas, my true love sent to me: 
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,

Using my functions on xmas2 dataset

In [524]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")

In [525]:
##ChatGPT helped me apply the sing_day function through days 1 through 12 to print out the entire lyrics of the new 12 Days of Christmas song with a seperation between each day. 
results2 = map(lambda day: sing_day(xmas2, day, 'Full.Phrase'), range(1, 13))
results2 = list(results2)
print("\n\n".join(results2))

On the first day of Christmas, my true love sent to me: 
an email from Cal Poly.

On the second day of Christmas, my true love sent to me: 
two meal points,
and an email from Cal Poly.

On the third day of Christmas, my true love sent to me: 
three lost pens,
two meal points,
and an email from Cal Poly.

On the fourth day of Christmas, my true love sent to me: 
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the fifth day of Christmas, my true love sent to me: 
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the sixth day of Christmas, my true love sent to me: 
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the seventh day of Christmas, my true love sent to me: 
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly